In [2]:
import json
import re
import pandas as pd
import neo4j


In [25]:
NEO4J_URI = "neo4j+s://023fca8e.databases.neo4j.io:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "XopDQ_W2ToQfo9fdgob8Zx4-piYAOK-qt26Ses0CBK0"
driver = neo4j.GraphDatabase.driver((NEO4J_URI), auth=(NEO4J_USERNAME, NEO4J_PASSWORD))


def find_simple_path(start_id, end_id, driver):
    with driver.session() as session:
        result = session.run("""
        MATCH path = (start:Token {id: $startId})-[:NEXT*..3]->(end:Token {id: $endId})
        RETURN path
        LIMIT 1
        """, {"startId": start_id, "endId": end_id})
        
        return [record["path"] for record in result]

In [26]:
# Example usage
start_node_id = 150
end_node_id = 200
path = find_simple_path(start_node_id, end_node_id, driver)

In [27]:
print(path)

[]


In [7]:
# Step 2: Extract relevant data for a specific question and model
def get_model_data(data, question, model_name):
    # Navigate through the dictionary to get to the relevant data
    # This assumes the data structure is {question: {model_name: data}}
    return data.get(question, {}).get(model_name)

In [5]:
with open("/p/llmreliability/llm_reliability/notebooks/combined_agent_responses.json", "r") as file:
    new_combined_data = json.load(file)

with open("/p/llmreliability/llm_reliability/experiments/truthful_qa/phi2_agent_responses.json", "r") as file:
    phi2_data = json.load(file)

merged_data = {}
for key in new_combined_data:
    merged_data[key] = {**new_combined_data[key], **phi2_data.get(key, {})}

with open("combined_agent_responses.json", "w") as file:
    json.dump(merged_data, file, indent=4)

In [5]:
from openai import OpenAI

In [6]:
NEO4J_URI = "neo4j+s://023fca8e.databases.neo4j.io:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "XopDQ_W2ToQfo9fdgob8Zx4-piYAOK-qt26Ses0CBK0"
agent = GPT35Agent("gpt-3.5-turbo", NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

In [7]:
chat = agent.start_chat("What is your name?")

In [13]:
print(chat.choices[0].logprobs.content[0])

ChatCompletionTokenLogprob(token='I', bytes=[73], logprob=-0.013185526, top_logprobs=[])


In [14]:
token = chat.choices[0].logprobs.content[0].token
logprob = chat.choices[0].logprobs.content[0].logprob
print(f"Token: {token}, logprob: {logprob}")

Token: I, logprob: -0.013185526


In [24]:
token_logprob = [{count: {value.token: value.logprob}} for count, value in enumerate(chat.choices[0].logprobs.content)]
print(token_logprob)

# {count:value.token for count, value in enumerate(chat.choices[0].logprobs.content)} 
# print(token_dict)

[{0: {'I': -0.013185526}}, {1: {' am': -0.006949799}}, {2: {' an': -0.029553989}}, {3: {' AI': -0.04122343}}, {4: {' language': -0.10031774}}, {5: {' model': -1.6240566e-06}}, {6: {' developed': -0.7675771}}, {7: {' by': -2.7729659e-05}}, {8: {' Open': -4.9617593e-06}}, {9: {'AI': -1.7835755e-05}}, {10: {',': -0.5865378}}, {11: {' and': -0.7668359}}, {12: {' I': -0.05906403}}, {13: {' do': -1.7719508}}, {14: {' not': -1.6240566e-06}}, {15: {' have': -0.042758908}}, {16: {' a': -0.0010332842}}, {17: {' personal': -0.12784202}}, {18: {' name': -0.0021756007}}, {19: {'.': -0.010752903}}, {20: {' You': -0.20428574}}, {21: {' can': -0.06249281}}, {22: {' refer': -0.97949636}}, {23: {' to': -5.5122365e-07}}, {24: {' me': -1.1994775e-05}}, {25: {' as': -0.00021378823}}, {26: {' Open': -0.50302887}}, {27: {'AI': -1.0564331e-05}}, {28: {' Assistant': -0.14758372}}, {29: {'.': -0.03794623}}, {30: {' How': -0.042728756}}, {31: {' can': -0.16591592}}, {32: {' I': -2.220075e-06}}, {33: {' assist': 

In [5]:
message = agent.give_response(chat)